In [1]:
import os

os.chdir('/content')
!git clone https://github.com/tienan-su/Masterarbeit.git

Cloning into 'Masterarbeit'...
remote: Enumerating objects: 66, done.
remote: Counting objects: 100% (66/66), done.
remote: Compressing objects: 100% (62/62), done.
remote: Total 66 (delta 26), reused 0 (delta 0), pack-reused 0 (from 0)
Receiving objects: 100% (66/66), 7.23 MiB | 4.45 MiB/s, done.
Resolving deltas: 100% (26/26), done.


In [3]:
%cd /content/Masterarbeit
!ls

/content/Masterarbeit
era5_dataset_pre.py			  training_wind_psr_lstm_era5.py
function				  wind_dataset_preparation.py
README.md				  wind_deep_simulation_framework.py
Results_2020_REMix_ReSTEP_hourly_REF.csv  wind_loss.py
siren_modules.py			  wind_trainer.py


In [11]:
from google.colab import files
uploaded = files.upload()

Saving Germany_2020_with_ws.nc to Germany_2020_with_ws.nc


In [8]:
!apt-get install -y libproj-dev proj-data proj-bin libgeos-dev
!pip install Cartopy
!pip install netCDF4
!pip install torch-scatter -f https://data.pyg.org/whl/torch-2.2.0+cpu.html
!pip install torch-sparse -f https://data.pyg.org/whl/torch-2.2.0+cpu.html
!pip install torch-geometric

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
libgeos-dev is already the newest version (3.12.1-1~jammy0).
libgeos-dev set to manually installed.
libproj-dev is already the newest version (9.3.1-1~jammy0).
libproj-dev set to manually installed.
proj-data is already the newest version (9.3.1-1~jammy0).
proj-data set to manually installed.
The following NEW packages will be installed:
  proj-bin
0 upgraded, 1 newly installed, 0 to remove and 35 not upgraded.
Need to get 205 kB of archives.
After this operation, 521 kB of additional disk space will be used.
Get:1 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy/main amd64 proj-bin amd64 9.3.1-1~jammy0 [205 kB]
Fetched 205 kB in 1s (150 kB/s)
Selecting previously unselected package proj-bin.
(Reading database ... 126435 files and directories currently installed.)
Preparing to unpack .../proj-bin_9.3.1-1~jammy0_amd64.deb ...
Unpacking proj-bin (9.3.1-1~jammy0) ...
Setting up proj

In [6]:
import numpy as np
import sys
import os

def setting_directory(depth):
    current_dir = os.path.abspath(os.getcwd())
    root_dir = current_dir
    for i in range(depth):
        root_dir = os.path.abspath(os.path.join(root_dir, os.pardir))
    sys.path.append(os.path.dirname(root_dir)) # Add the parent directory to the path
    sys.path.append(os.path.join(root_dir, 'function')) # Add the 'function' subdirectory to the path
    return root_dir

root_dir = setting_directory(0)

from pathlib import Path
import torch
from scipy import linalg
import torch.nn as nn
import torch.nn.init as init
from siren_modules import Siren

import warnings
import time

from tqdm import tqdm
import matplotlib.pyplot as plt
import scipy.io
from scipy.interpolate import griddata
from mpl_toolkits.axes_grid1 import make_axes_locatable
from sklearn.model_selection import train_test_split

warnings.filterwarnings("ignore")

# Fix randomness
np.random.seed(1234)
torch.manual_seed(7)

# CUDA support
if torch.cuda.is_available():
    device = torch.device("cuda")
else:
    device = torch.device("cpu")

In [10]:
from wind_dataset_preparation_psr_era5 import (
    extract_pressure_for_germany,
    extract_wind_speed_for_germany,
    extract_temperature_for_germany,
    load_real_wind_csv,
    interpolate_wind_speed,
    loading_wind,
    interpolate_pressure,
    interpolate_temperature,
    scale_interpolated_data,
    combine_data,
    repeat_target_points,
    scale_target_points
    )

from wind_dataset_preparation import LSTMDataPreparation, CNNBiLSTMDataPreparation
from wind_deep_simulation_framework import LSTMDeepModel, CNN_BiLSTM
from wind_loss import wind_loss_func
from wind_trainer import LSTMTrainer, CNNBiLSTMTrainer


In [12]:
# Example usage
nc_file_path = 'Germany_2020_with_ws.nc'
csv_file_path = 'Results_2020_REMix_ReSTEP_hourly_REF.csv'

# Extract pressure data
pressure_data, grid_lats, grid_lons = extract_pressure_for_germany(nc_file_path)

print(f"Shape of extracted pressure: {pressure_data.shape}")
print(f"Sample of extracted pressure (first 5 time steps, first 5 locations):\n{pressure_data[:5, :5]}")


wind_speeds, grid_lats, grid_lons = extract_wind_speed_for_germany(nc_file_path)


print(f"Shape of extracted wind speed: {wind_speeds.shape}")
print(f"Sample of extracted wind speed (first 5 time steps, first 5 locations):\n{wind_speeds[:5, :5]}")


temperature_data, grid_lats, grid_lons = extract_temperature_for_germany(nc_file_path)

print("Shape of extracted temperature:", temperature_data.shape)
print("First 5 values:\n", temperature_data[:, :5])
print("First time step temperature map:\n", temperature_data[0])
print(f"Sample of extracted temperature (first 5 time steps, first 5 locations):\n{temperature_data[:5, :5]}")


target_points = load_real_wind_csv(csv_file_path)

scaled_unix_time_array, filtered_x_y, filtered_wind_power = loading_wind()

interpolated_wind_speeds = interpolate_wind_speed(wind_speeds, grid_lats, grid_lons, target_points)

interpolated_pressure = interpolate_pressure(pressure_data, grid_lats, grid_lons, target_points)

# Interpolate temperature data
interpolated_temperature = interpolate_temperature(temperature_data, grid_lats, grid_lons, target_points)
interpolated_temperature = interpolated_temperature - 273.15 # now in °C


scaled_wind_speeds = scale_interpolated_data(interpolated_wind_speeds)

scaled_pressure = scale_interpolated_data(interpolated_pressure)

scaled_temperature = scale_interpolated_data(interpolated_temperature)

scaled_wind_power = scale_interpolated_data(filtered_wind_power)

scaled_target_points = scale_target_points(target_points)

# Number of time steps (from scaled_wind_speeds)
num_time_steps = scaled_wind_speeds.shape[0]
repeated_scaled_target_points = repeat_target_points(scaled_target_points, num_time_steps)

print(f"Shape of repeated_scaled_target_points: {repeated_scaled_target_points.shape}")


Shape of extracted pressure: (2928, 28574)
Sample of extracted pressure (first 5 time steps, first 5 locations):
[[ 99258.5    99258.5    99258.5   100050.    100050.   ]
 [ 99230.     99230.     99230.    100014.    100014.   ]
 [ 99224.69   99224.69   99224.69  100011.69  100011.69 ]
 [ 99276.375  99276.375  99276.375 100062.375 100062.375]
 [ 99305.19   99305.19   99305.19  100094.69  100094.69 ]]
Shape of extracted wind speed: (2928, 28574)
Sample of extracted wind speed (first 5 time steps, first 5 locations):
[[1.7151978  1.7151978  1.7151978  2.0236347  2.0236347 ]
 [1.7679368  1.7679368  1.7679368  1.554989   1.554989  ]
 [1.1141016  1.1141016  1.1141016  1.4631282  1.4631282 ]
 [0.7916001  0.7916001  0.7916001  0.60546654 0.60546654]
 [1.3422633  1.3422633  1.3422633  1.6280413  1.6280413 ]]
Shape of extracted temperature: (2928, 28574)
First 5 values:
 [[271.61548 271.61548 271.61548 271.82788 271.82788]
 [271.48535 271.48535 271.48535 270.19336 270.19336]
 [269.93823 269.938

In [13]:
# Combine the data
combined_array = combine_data(scaled_target_points, scaled_unix_time_array,
                              scaled_wind_speeds,
                              scaled_pressure,
                              scaled_temperature,
                              scaled_wind_power)

# Check the shape of the combined array
print(combined_array.shape)

type(combined_array)


Shape of combined_array: (679296, 7)
(679296, 7)


numpy.ndarray

In [14]:
wind_dataset_instance = LSTMDataPreparation(combined_array[:,:6], combined_array[:,6:])

"""
test_data_size = 0.05
"""

test_data_size = 0.98


x_train_seq, u_train_seq, train_loader, test_loader = wind_dataset_instance.prepare_data_random(test_data_size)

In [15]:
# Dataset configuration

input_size = 6  # Number of input features
hidden_features = 64  # Number of RNN hidden units
hidden_layers = 4  # Number of RNN layers
output_size = 1  # Number of output features
learning_rate = 1e-3


In [16]:
lstm_deep_model_instance = LSTMDeepModel(input_size, hidden_features,
                                       hidden_layers, output_size,
                                       learning_rate, )

# Print dataset shapes for verification
print(f"Training sequences shape: {x_train_seq.shape}")


model_str, optim_adam, scheduler = lstm_deep_model_instance.run()


"""
num_epochs = 25000
"""
num_epochs = 10


Train_inst = LSTMTrainer(
    model_str,
    num_epochs=num_epochs,
    optim_adam=optim_adam,
    scheduler=scheduler,

)

loss_func_list = Train_inst.train_func(train_loader, test_loader)



Training sequences shape: torch.Size([13575, 10, 6])
